In [1]:
import pandas as pd
import html
import re

In [2]:
d = pd.read_csv("2018-E-c-En-train.txt",sep="\t")
d = d.drop(['ID'],axis=1)
def removeXA0(Tweet):
    return Tweet.replace(u'\xa0', u' ')
def removeLineFeed(Tweet):
    return Tweet.replace(u'\\n',u' ')
def removeCarriageReturn(Tweet):
    return Tweet.replace(u'\\r',u' ')
def removeCitation(Tweet):
    return re.sub(r"(?:@[A-Za-z0-9_]+)","",Tweet)
#def deEmojify(inputString):
    #return inputString.encode('ascii', 'ignore').decode('ascii')

Replace emojis with text translations given in emoji list (emojis: dictionary of translated emojis). Will probably add some extra slash characters so other processing steps can be done after this one. 

In [8]:
def emojify(text, emojis):
	text = str(text.encode('unicode-escape'))[2:-1]
	if '\\u' in text.lower():
		text = text.replace('\\\\U' , '\\\\u')
		text = text.replace('\\\\u' , ' \\\\u')
		words = text.split(' ')
		for word in words:
			if '\\u' in word:
				if word in emojis.keys():
					words[words.index(word)] = emojis[word]
				elif word[0:11] in emojis:
					word_1 = word[11:len(word)]
					words[words.index(word)] = emojis[word[0:11]] + ' ' + word_1
				elif word[0:7] in emojis:
					word_1 = word[7:len(word)]
					words[words.index(word)] = emojis[word[0:7]] + ' ' + word_1
		return ' '.join(words)
	return text

In [9]:
emojis = {}
with open('data/emojilist5.csv', 'r') as f:
    for line in f:
        unic=line.split(',')[0].lower()
        trans=line.split(',')[1]
        emojis[unic]=trans
d.Tweet = d.Tweet.apply(emojify, args=(emojis,))

### There is no in the train dataset:
 - HTML tags
 - URLs
 - \t

In [151]:
d.Tweet = d.Tweet.apply(removeXA0)
d.Tweet = d.Tweet.apply(removeLineFeed)
d.Tweet = d.Tweet.apply(removeCarriageReturn)
d.Tweet = d.Tweet.apply(removeCitation)
d.Tweet = d.Tweet.apply(deEmojify)

In [191]:
k = d.iloc[21].Tweet.replace("&amp;","")
print(k)

#NewYork: Several #Baloch  Indian activists hold demonstrations outside  headquarters demanding Pak to stop exporting #terror into India


In [189]:
s = html.escape( """& < >""" )   # s = "&amp; &lt; &gt;"

In [154]:
d.iloc[21].Tweet

'#NewYork: Several #Baloch &amp; Indian activists hold demonstrations outside  headquarters demanding Pak to stop exporting #terror into India'

In [153]:
d

,Tweet,anger,anticipation,disgust,fear,joy,love,optimism,pessimism,sadness,surprise,trust
0,Worry is a down payment on a problem you may n...,0,1,0,0,0,0,1,0,0,0,1
1,Whatever you decide to do make sure it makes y...,0,0,0,0,1,1,1,0,0,0,0
2,it also helps that the majority of NFL coach...,1,0,1,0,1,0,1,0,0,0,0
3,Accept the challenges so that you can literall...,0,0,0,0,1,0,1,0,0,0,0
4,My roommate: it's okay that we can't spell bec...,1,0,1,0,0,0,0,0,0,0,0
5,No but that's so cute. Atsu was probably shy a...,0,0,0,0,1,0,0,0,0,0,0
6,Do you think humans have the sense for recogni...,0,1,0,0,0,0,0,1,0,0,0
7,Rooneys fucking untouchable isn't he? Been fuc...,1,0,1,0,0,0,0,0,0,0,0
8,it's pretty depressing when u hit pan on ur fa...,0,0,1,0,0,0,0,0,1,0,0
9,but your pussy was weak from what I heard so ...,1,0,1,0,0,0,0,0,0,0,0
